In [ ]:
!pip install pycoingecko

In [ ]:
import pandas as pd
mport requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=4)
cg = CoinGeckoAPI()

In [ ]:
def find_id(row):
    if row['Ticker'].lower() in coin_df.symbol.values:
        temp_df = coin_df[coin_df['symbol'] == row['Ticker'].lower()]
        if len(temp_df) > 1:
            listing = row['listings'].lower()
            if listing in temp_df.id.values:
                temp_df = temp_df[temp_df['id'] == listing]
            elif listing.title() in temp_df.name.values:
                temp_df = temp_df[temp_df['name'] == listing.title()]
        return temp_df['id'].item()

def get_listing_price(row, currency):
    r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/history?date={row['listing date']}")
    time.sleep(1)
    try:
        json = r.json()
    except ValueError:
        print(r.text)
    if 'market_data' not in json:
        return "added to coingecko after coinbase consideration"
        print(json, row['ids'])
    prices = json['market_data']['current_price'][currency]
    return prices

def get_specified_price(row, currency, date):
    r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/history?date={date}")
    time.sleep(1)
    try:
        json = r.json()
    except ValueError:
        print(r.text)
        raise ValueError
    if 'market_data' not in json:
        return "added to coingecko after coinbase consideration"
        print(json, row['ids'])
    prices = json['market_data']['current_price'][currency]
    return prices

def get_market_cap(row):
    r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/history?date={row['listing date']}")
    time.sleep(1)
    try:
        json = r.json()
    except ValueError:
        print(r.text)
    if 'market_data' not in json:
        print(json, row['ids'])
        return "added to coingecko after coinbase consideration"
    prices = json['market_data']['market_cap']['usd']
    return prices

def get_specified_market_cap(row, date):
    r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/history?date={date}")
    time.sleep(1)
    try:
        json = r.json()
    except ValueError:
        print(r.text)
    if 'market_data' not in json:
        print(json, row['ids'])
        return "added to coingecko after coinbase consideration"
    prices = json['market_data']['market_cap']['usd']
    return prices

In [ ]:
pd.set_option('max_columns', None)

In [ ]:
coin_df = pd.DataFrame.from_records(cg.get_coins_list())

In [ ]:
gecko_df = pd.read_csv('coingecko_listing.csv')

In [ ]:
gecko_df.dropna(axis = 1, inplace=True, how='all')
gecko_df.dropna(axis = 0, inplace=True, how = 'all')

In [ ]:
gecko_df['ids'] = gecko_df.apply(find_id,axis=1)

gecko_df.loc[gecko_df['Ticker'] == 'SUKU','ids'] = coin_df.loc[coin_df['symbol'] == 'SUKU']['id'].item()

gecko_df.loc[gecko_df['Ticker'] == 'WLUNA','ids'] =coin_df.loc[coin_df['id'] == 'wrapped-terra']['id'].item()

In [ ]:
gecko_df.loc[gecko_df['Ticker'] == 'CGLD','ids'] = 'celo'

In [ ]:
gecko_df['Price at Listing USD'] = gecko_df.apply(lambda x: get_listing_price(x, 'usd'),axis=1)

In [ ]:
gecko_df['Market Cap at listing USD'] = gecko_df.apply(get_market_cap,axis=1)                                        

In [ ]:
gecko_df['Price at Listing BTC'] = gecko_df.apply(lambda x: get_listing_price(x, 'btc'),axis=1)                                        

In [ ]:
gecko_df['Price at Listing ETH'] = gecko_df.apply(lambda x: get_listing_price(x, 'eth'),axis=1)

In [ ]:
gecko_df['Price at 1/09 USD'] = gecko_df.apply(lambda x: get_specified_price(x, 'usd', '09-01-2022'),axis=1)

In [ ]:
gecko_df['Price at 1/09 BTC'] = gecko_df.apply(lambda x: get_specified_price(x, 'btc', '09-01-2022'),axis=1)

In [ ]:
gecko_df['Price at 1/09 ETH'] = gecko_df.apply(lambda x: get_specified_price(x, 'eth', '09-01-2022'),axis=1)

In [ ]:
gecko_df['Current Market Cap'] = gecko_df.apply(lambda x: get_specified_market_cap(x, '09-01-2022'), axis=1)

In [ ]:
def price_prior(row):
    date = row['listing date']
    date = date.split('-')
    date = date[0] + '-' +date[1] + '-' + str(int(date[2]) - 1)
    print(date)
    r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/history?date={date}")
    time.sleep(1)
    try:
        json = r.json()
    except ValueError:
        print(r.text)
    if 'market_data' not in json:
        return "No previous year available"
        print(json, row['ids'])
    prices = json['market_data']['current_price']['usd']
    return prices
gecko_df['previous year price'] = gecko_df.apply(price_prior, axis=1)

In [ ]:
gecko_df.to_csv('prices_for_coins.csv')

In [ ]:
#check for cells with no id, name, price, or ticker
is_NaN = gecko_df.isnull()

row_has_NaN = is_NaN.any(axis=1)

rows_with_NaN = gecko_df[row_has_NaN]


print(rows_with_NaN)

In [ ]:
gecko_df

In [ ]:
#check unlisted coins
not_listed_df = pd.read_csv('not_listed.csv')

In [ ]:
not_listed_df.dropna(axis=0, how='all', subset= ['name'], inplace=True)

In [ ]:
not_listed_df.rename(columns={'Consideration date':'listing date', 'name':'ids'}, inplace=True)

In [ ]:
not_listed_df.dropna(axis=1, how="all", inplace=True)

In [ ]:
not_listed_df.drop(axis=1,labels=['Unnamed: 0','Unnamed: 13','Unnamed: 14'], inplace=True)

In [ ]:
def fix_date(date):
    if '/' in date:
        date = date.split('/')
        date = date[0] +'-'+date[1]+'-20'+date[2]
    return date

not_listed_df['listing date'] = not_listed_df['listing date'].apply(fix_date)

In [ ]:
not_listed_df['Price at listing'] = not_listed_df.apply(lambda x: get_listing_price(x, 'usd'),axis=1)

not_listed_df['Price at listing BTC'] = not_listed_df.apply(lambda x: get_listing_price(x, 'btc'),axis=1)

not_listed_df['Price at listing ETH'] = not_listed_df.apply(lambda x: get_listing_price(x, 'eth'),axis=1)

In [ ]:
not_listed_df['Price at 1/09 USD'] = not_listed_df.apply(lambda x: get_specified_price(x, 'usd', '09-01-2022'),axis=1)

not_listed_df['Price at 1/09 BTC'] = not_listed_df.apply(lambda x: get_specified_price(x, 'btc', '09-01-2022'),axis=1)

not_listed_df['Price at 1/09 ETH'] = not_listed_df.apply(lambda x: get_specified_price(x, 'eth', '09-01-2022'),axis=1)

not_listed_df['Current Market Cap'] = not_listed_df.apply(lambda x: get_specified_market_cap(x, '09-01-2022'),axis=1)

In [ ]:
df = pd.concat([gecko_df, not_listed_df])
df.to_csv('current_coin_prices.csv')

In [ ]:
def get_earliest_data_price(row, currency):
    if row['Price at listing'] == 'added to coingecko after coinbase consideration':
        r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/market_chart?vs_currency={currency}&days=max")
        time.sleep(0.5)
        json = r.json()
        if 'prices' not in json:
            pp.pprint(json)
        prices = json['prices'][0][1]
        return prices
    else:
        return None
    
def get_earliest_data_market_cap(row):
    if row['market cap at consideration'] == 'added to coingecko after coinbase consideration':
        r = requests.get(f"https://api.coingecko.com/api/v3/coins/{row['ids']}/market_chart?vs_currency={'usd'}&days=max")
        time.sleep(0.5)
        json = r.json()
        if 'market_caps' not in json:
            pp.pprint(json)
        prices = json['market_caps'][0][1]
        return prices
    else:
        return None
    


In [ ]:
not_listed_df['earliest listed price'] = not_listed_df.apply(lambda x: get_earliest_data_price(x, 'usd'), axis=1)
not_listed_df['earliest listed price BTC'] = not_listed_df.apply(lambda x: get_earliest_data_price(x, 'btc'), axis=1)
not_listed_df['earliest listed price ETH'] = not_listed_df.apply(lambda x: get_earliest_data_price(x, 'eth'), axis=1)

In [ ]:
not_listed_df['market cap at consideration'] = not_listed_df.apply(get_market_cap, axis=1)

not_listed_df['market cap earliest listed'] = not_listed_df.apply(get_earliest_data_market_cap, axis=1)

not_listed_df['current market cap'] = not_listed_df.apply(get_specified_market_cap,axis=1)

In [ ]:
not_listed_df.dropna(axis=1, how='all')

In [ ]:
not_listed_df.to_csv('not_listed_added_info.csv')